In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# !pip install torchvision

In [3]:
from torchvision import transforms
import torch.nn as nn
import numpy as np
import os
from PIL import Image
from torch.utils.data import DataLoader , Dataset , Subset
import torch
from sklearn.model_selection import train_test_split
from HOG import HOG
from Softmax import Softmax


# Lets Start with the loading the dataset for Training our Model

In [4]:
import kagglehub
path = kagglehub.dataset_download("maciejgronczynski/vehicle-classification-dataset")
print("Path to dataset files:", path)

In [7]:

class CreateDataset(Dataset):
    def __init__(self,DataSetPath,folders,HOG):
        imagesPath = list()
        labels = list()
        for folder in folders:
            for root,subdirs,files in os.walk(os.path.join(DataSetPath,folder)):
                imagesPath.extend([os.path.join(root , file) for file in files])
                labels.extend([folder for _ in files])        
        self.imagesPath = imagesPath
        self.labels = labels
        self.HOG = HOG
        self.LabelToIdxMapping = {label:idx for idx,label in enumerate(folders)}
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self,index):
        image = Image.open(self.imagesPath[index]).convert('L')
        hog_image = self.HOG.GetFeatures(image)
        hog_tensor = torch.tensor(hog_image, dtype=torch.float32)
        # Encode the label to a tensor
        label = self.LabelToIdxMapping[self.labels[index]]
        label_tensor = torch.tensor(label, dtype=torch.long)
        return {"image": hog_tensor, "Label": label_tensor}


In [19]:
# Initialize the dataset
DataSetPath = "/kaggle/input/vehicle-classification-dataset/dataset"
folders = ["cars", "planes", "trains"]
hogFeatures = HOG(9, 8, 2)
datasets = CreateDataset(DataSetPath=DataSetPath, folders=folders, HOG=hogFeatures)

indices = np.arange(len(datasets))  
labels = [datasets[i]["Label"] for i in indices]

train_indices, test_indices = train_test_split(
    indices, test_size=0.2, random_state=42, stratify=labels
)
train_dataset = Subset(datasets, train_indices)
test_dataset = Subset(datasets, test_indices)

train_loader = DataLoader(dataset=train_dataset ,shuffle=True)
val_loader = DataLoader(dataset=test_dataset, shuffle=False)

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
model = Softmax(inputNeurons=8100, hlayer1=2100, outputs=3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, mode='triangular')
epochs = 50
loss_history = []
max_acc = 0
best_model_path = "softmax_HOG.pth"

In [24]:
for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0

    for batch in train_loader:
        optimizer.zero_grad()
        images = batch["image"].to(device)
        labels = batch["Label"].to(device)
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        # scheduler.step()
        epoch_loss += loss.item()
    loss_history.append(epoch_loss)
    model.eval()
    correctCount = 0
    totalSamples = 0

    with torch.no_grad():  
        for batch in val_loader:
            images = batch["image"].to(device)
            labels = batch["Label"].to(device)
            output = model(images)
            predictions = torch.argmax(output, dim=1)
            correctCount += (predictions == labels).sum().item()
            totalSamples += labels.size(0)

    accuracy = (correctCount / totalSamples) * 100

    
    if accuracy > max_acc:
        max_acc = accuracy
        torch.save(model.state_dict(), best_model_path)


    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

Epoch 1/50, Loss: 0.9470, Accuracy: 75.83%
Epoch 2/50, Loss: 0.5496, Accuracy: 89.58%
Epoch 3/50, Loss: 0.3610, Accuracy: 73.75%
Epoch 4/50, Loss: 0.2381, Accuracy: 94.17%
Epoch 5/50, Loss: 0.1764, Accuracy: 94.17%
Epoch 6/50, Loss: 0.1446, Accuracy: 93.33%
Epoch 7/50, Loss: 0.1264, Accuracy: 93.75%
Epoch 8/50, Loss: 0.1092, Accuracy: 95.00%
Epoch 9/50, Loss: 0.0525, Accuracy: 92.08%
Epoch 10/50, Loss: 0.0454, Accuracy: 95.00%
Epoch 11/50, Loss: 0.0516, Accuracy: 91.67%
Epoch 12/50, Loss: 0.0158, Accuracy: 95.00%
Epoch 13/50, Loss: 0.0230, Accuracy: 94.17%
Epoch 14/50, Loss: 0.0303, Accuracy: 94.17%
Epoch 15/50, Loss: 0.0355, Accuracy: 95.00%
Epoch 16/50, Loss: 0.0044, Accuracy: 95.00%
Epoch 17/50, Loss: 0.0272, Accuracy: 92.92%
Epoch 18/50, Loss: 0.0036, Accuracy: 95.42%
Epoch 19/50, Loss: 0.0009, Accuracy: 95.42%
Epoch 20/50, Loss: 0.0006, Accuracy: 95.00%
Epoch 21/50, Loss: 0.0005, Accuracy: 95.42%
Epoch 22/50, Loss: 0.0004, Accuracy: 95.42%
Epoch 23/50, Loss: 0.0003, Accuracy: 95.4

In [1]:
from huggingface_hub import Repository

REPO_URL = "https://huggingface.co/ibrahimsultan/HOG_Softmax_image_classifier"
LOCAL_PATH = "HOG_Softmax_image_classifier"  # Local directory to clone the repo
MODEL_FILE = "softmax_HOG.pth"  # Path to the model file

repo = Repository(local_dir=LOCAL_PATH, clone_from=REPO_URL)
repo.git_pull()  # Pull the latest changes

repo.lfs_track([MODEL_FILE])  # Track large files with Git LFS
repo.push_to_hub(commit_message="Upload HOG_Softmax model", files_to_push=[MODEL_FILE])

print("Model uploaded successfully!")


c:\Users\hp\Desktop\Machine Learning\ComputerVision\HOG-feature-for-vehicle-Classification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hp\Desktop\Machine Learning\ComputerVision\HOG-feature-for-vehicle-Classification\.venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/ibrahimsultan/HOG_Softmax_image_classifier into local empty directory.


TypeError: Repository.push_to_hub() got an unexpected keyword argument 'files_to_push'